В этом файле собран весь код по подготовке данных, обучению модели, тестировании модели и её сохранении. 

Сначала разделим все файлы по инструментам

In [ ]:
import os
import pretty_midi

def get_instrument_name(program_number):
    general_midi_instruments = {
        0: 'Acoustic Grand Piano',
        1: 'Bright Acoustic Piano',
        6: 'Harpsichord',
        8: 'Celesta',
        9: 'Glockenspiel',
        11: 'Vibraphone',
        12: 'Marimba',
        13: 'Xylophone',
        14: 'Tubular Bells',
        15: 'Dulcimer',
        17: 'Percussive Organ',
        19: 'Church Organ',
        21: 'Accordion',
        22: 'Harmonica',
        24: 'Acoustic Guitar _nylon',
        25: 'Acoustic Guitar _steel',
        31: 'Guitar Harmonics',
        32: 'Acoustic Bass',
        40: 'Violin',
        41: 'Viola',
        42: 'Cello',
        43: 'Contrabass',
        44: 'Tremolo Strings',
        45: 'Pizzicato Strings',
        46: 'Orchestral Harp',
        47: 'Timpani',
        56: 'Trumpet',
        57: 'Trombone',
        58: 'Tuba',
        59: 'Muted Trumpet',
        60: 'French Horn',
        65: 'Alto Sax',
        67: 'Baritone Sax',
        68: 'Oboe',
        69: 'English Horn',
        70: 'Bassoon',
        71: 'Clarinet',
        72: 'Piccolo',
        73: 'Flute',
        75: 'Pan Flute',
        79: 'Ocarina',
        105: 'Banjo',
        108: 'Kalimba',
        109: 'Bagpipe',
        110: 'Fiddle',
        112: 'Tinkle Bell',
    }
    return general_midi_instruments.get(program_number, f'Unknown_Instrument_{program_number}')

def process_midi_file(midi_path, output_folder):
    try:
        midi_data = pretty_midi.PrettyMIDI(midi_path)
        
        os.makedirs(output_folder, exist_ok=True)

        for instrument in midi_data.instruments:
            program_number = instrument.program
            instrument_name = get_instrument_name(program_number)
            
            safe_name = "".join(c if c.isalnum() or c in (' ', '_') else '_' for c in instrument_name)
            output_path = os.path.join(output_folder, f'{safe_name}.mid')
            
            if os.path.exists(output_path):
                existing_midi = pretty_midi.PrettyMIDI(output_path)
                # Находим соответствующий инструмент или создаем новый
                found = False
                for existing_instr in existing_midi.instruments:
                    if existing_instr.program == program_number:
                        existing_instr.notes.extend(instrument.notes)
                        found = True
                        break
                
                if not found:
                    new_instrument = pretty_midi.Instrument(program=program_number)
                    new_instrument.notes = instrument.notes.copy()
                    existing_midi.instruments.append(new_instrument)
                
                existing_midi.write(output_path)
            else:
                # Если нет файла для инструмента - здесь создаем его
                new_midi = pretty_midi.PrettyMIDI()
                new_instrument = pretty_midi.Instrument(program=program_number)
                new_instrument.notes = instrument.notes.copy()
                new_midi.instruments.append(new_instrument)
                new_midi.write(output_path)
            
            print(f'Processed: {output_path}')

    except Exception as e:
        print(f"Ошибка при обработке файла {midi_path}: {e}")

def process_all_midi(input_folder, output_folder):
    if not os.path.exists(input_folder):
        print(f"Папка {input_folder} не существует!")
        return
    
    os.makedirs(output_folder, exist_ok=True)
    
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith('.mid'):
                midi_path = os.path.join(root, file)
                process_midi_file(midi_path, output_folder)

# Сначала используем на lmd_full/a, потом на lmd_full/b, чтобы в обучающей и тестовой выборках были различные данные
midi_folder_a = 'lmd_full/a'
midi_folder_b = 'lmd_full/b'
output_folder_a = os.path.join(midi_folder_a, 'instruments')
output_folder_b = os.path.join(midi_folder_b, 'instruments')
def midi_to_instruments():
    process_all_midi(midi_folder_a, output_folder_a)
    process_all_midi(midi_folder_b, output_folder_b)
    


Теперь нужно очистить файлы от пауз. Воспользуемся первой очисткой от пауз

In [ ]:
import os
import pretty_midi

INSTRUMENT_NOTES = {
    'Accordion': (40, 108),
    'Acoustic Bass': (28, 67),
    'Acoustic Guitar _nylon': (52, 91),
    'Acoustic Guitar _steel': (52, 91),
    'Acoustic Grand Piano': (21, 108),
    'Alto Sax': (58, 90),
    'Bagpipe': (62, 81),
    'Banjo': (55, 79),
    'Baritone Sax': (58, 90),
    'Bassoon': (35, 77),
    'Bright Acoustic Piano': (21, 108),
    'Celesta': (60, 108),
    'Cello': (36, 81),
    'Church Organ': (36, 96),
    'Clarinet': (52, 91),
    'Contrabass': (28, 71),
    'Dulcimer': (48, 96),
    'English Horn': (48, 83),
    'Fiddle': (55, 105),
    'Flute': (60, 96),
    'French Horn': (35, 77),
    'Glockenspiel': (77, 108),
    'Guitar Harmonics': (59, 101),
    'Harmonica': (54, 96),
    'Harpsichord': (29, 89),
    'Kalimba': (50, 89),
    'Marimba': (36, 96),
    'Muted Trumpet': (54, 84),
    'Oboe': (58, 89),
    'Ocarina': (55, 96),
    'Orchestral Harp': (29, 103),
    'Pan Flute': (36, 93),
    'Percussive Organ': (36, 96),
    'Piccolo': (74, 108),
    'Pizzicato Strings': (28, 100),
    'Timpani': (74, 93),
    'Tinkle Bell': (84, 115),
    'Tremolo Strings': (28, 100),
    'Trombone': (31, 77),
    'Trumpet': (60, 83),
    'Tuba': (24, 64),
    'Tubular Bells': (55, 84),
    'Vibraphone': (48, 96),
    'Viola': (48, 88),
    'Violin': (55, 105),
    'Xylophone': (53, 96)
}

def is_valid_instrument(instrument_name):
    return instrument_name in INSTRUMENT_NOTES

def remove_midi_pauses(midi_path, midi_folder, output_path=None):
    try:        
        midi = pretty_midi.PrettyMIDI(midi_folder+"/"+midi_path)
    
        instrument_name = midi_path.split('.')[0]
        
        all_notes = [note for inst in midi.instruments for note in inst.notes]
    
        if not all_notes:
            return False
    
        if not is_valid_instrument(instrument_name):
            print(f"Пропущен (невалидный инструмент): {instrument_name}")
            return False
    
        instrument_range = INSTRUMENT_NOTES[instrument_name]

        first_note_start = min(n.start for n in all_notes)
        last_note_end = max(n.end for n in all_notes)
    
        for instrument in midi.instruments:
            if not instrument.notes:
                continue
            
            if not instrument_range:
                continue
            
            valid_notes = []
            current_time = 0.0
            
            instrument.notes.sort(key=lambda x: x.start)
        
            time_shift = first_note_start
            current_time = 0.0
        
            new_notes = []
        
            total_duration = sum(note.end - note.start for note in instrument.notes)
        
            current_time = 0.0
            
            for note in instrument.notes:
                if instrument_range[0] <= note.pitch <= instrument_range[1]:
                    note_duration = note.end - note.start
                    note.start = current_time
                    note.end = current_time + note_duration
                    current_time += note_duration
                    valid_notes.append(note)
        
            instrument.notes = valid_notes
        
        if output_path is None:
            base, ext = os.path.splitext(midi_path)
            output_path = f"{midi_folder}/valid/{base}_no_pauses{ext}"
            
        midi.instruments = [inst for inst in midi.instruments if inst.notes]

        if not midi.instruments:
            print(f"Пропущен (нет валидных нот): {instrument_name}")
            return False

        midi.write(output_path)
        print(f"Успешно обработан: {instrument_name}")
        return True
    
    except Exception as e:
        print(f"Ошибка при обработке {midi_path}: {str(e)}")
        return False

def midi_detele_silence():
    for file in os.listdir(output_folder_a):
        if file.endswith('.mid'):
            remove_midi_pauses(file, output_folder_a)
            print("Changed "+file)
    for file in os.listdir(output_folder_b):
        if file.endswith('.mid'):
            remove_midi_pauses(file, output_folder_b)
            print("Changed "+file)

Теперь запускаем "нормализацию". А именно - сокращаем размеры файлов инструментов до 5 тысяч нот, за исключением скрипки

In [ ]:
import pretty_midi
import os

def crop_midi(midi_path):
    midi_data = pretty_midi.PrettyMIDI(midi_path)
    if len(midi_data.instruments) != 1:
        raise ValueError("Файл должен содержать ровно один инструмент")
    instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
    if instrument_name != 'Violin':
        max_notes = 10000
    instrument = midi_data.instruments[0]
    print(len(instrument.notes))
    if len(instrument.notes) > max_notes:
        instrument.notes = instrument.notes[:max_notes]
    temp_path = midi_path + ".tmp"
    try:
        midi_data.write(temp_path)
        os.replace(temp_path, midi_path)
        print(f"Файл успешно перезаписан: {midi_path}")
        
    except Exception as e:
        print(f"Ошибка при сохранении: {e}")
        if os.path.exists(temp_path):
            os.remove(temp_path)
            
            
midi_folder_a = "lmd_full/a/instruments"
midi_folder_b = "lmd_full/b/instruments"

def normalize(midi_folder):
    for root, dirs, files in os.walk(midi_folder):
        if not files:
            print(f"Папка {root} пуста")
            continue
        for i, file in enumerate(files):
            if not file.lower().endswith(('.mid', '.midi')):
                continue
            
            full_path = os.path.join(root, file)
            print(f"Обработка: {full_path}")
            crop_midi(full_path)
            
def normalize_midi_instrumemts():
    normalize(midi_folder_a)
    normalize(midi_folder_b)

Теперь нужно преобразовать в формат wav

In [ ]:
import subprocess
import os
import time

# Используется GeneralUser-GS.sf2 в качестве звукового шрифта
SOUNDFONT_PATH = os.path.abspath("GeneralUser-GS.sf2")
SAMPLE_RATE = 44100

def midi_to_wav_folder(midi_folder):
    output_dir = os.path.join(midi_folder, "wav")
    os.makedirs(output_dir, exist_ok=True)

    for file in os.listdir(midi_folder):
        if not file.lower().endswith(".mid"):
            continue

        midi_path = os.path.abspath(os.path.join(midi_folder, file))
        output_wav = os.path.abspath(os.path.join(output_dir, os.path.splitext(file)[0] + ".wav"))

        cmd = f'fluidsynth -ni -F "{output_wav}" -r {SAMPLE_RATE} "{SOUNDFONT_PATH}" "{midi_path}"'

        try:
            start = time.time()
            with subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True) as proc:
                for line in proc.stdout:
                    print("🧾", line.strip())
                proc.wait(timeout=60)
            duration = time.time() - start
        except subprocess.TimeoutExpired:
            proc.kill()
        except Exception as e:
            print(f"Ошибка: {file}: {e}")
def midi_to_wav():
    midi_to_wav_folder(midi_folder_a)
    midi_to_wav_folder(midi_folder_b)

Теперь создаем модель машинного обучения бинарного распознавания "скрипка" / "не скрипка"

Сначала разделим аудиофайлы на сегменты

In [ ]:
import librosa
import soundfile as sf
import os
import numpy as np

def split_audio_to_segments(file_path, segment_duration=1.0, overlap=0.5):
    y, sr = librosa.load(file_path, sr=22050)
    segment_samples = int(segment_duration * sr)
    step = int(segment_samples * (1 - overlap))
    
    segments = []
    for start in range(0, len(y) - segment_samples, step):
        segment = y[start:start + segment_samples]
        segments.append(segment)
    
    return segments

def augment_audio_segment(segment, sr=22050):
    augmented = []
    
    noise = 0.005 * np.random.randn(len(segment))
    augmented.append(segment + noise)
    
    augmented.append(librosa.effects.pitch_shift(segment, sr=sr, n_steps=2))
    augmented.append(librosa.effects.pitch_shift(segment, sr=sr, n_steps=-2))
    
    augmented.append(librosa.effects.time_stretch(segment, rate=0.8))
    augmented.append(librosa.effects.time_stretch(segment, rate=1.2))
    
    return augmented

def split_wav(input_path, output_dir, segment_length=3, overlap=0.1):
    y, sr = librosa.load(input_path, sr=None)
    segment_samples = int(segment_length * sr)
    step = int(segment_samples * (1 - overlap))
    
    os.makedirs(output_dir, exist_ok=True)
    
    for i, start in enumerate(range(0, len(y), step)):
        end = start + segment_samples
        segment = y[start:end]
        if len(segment) < segment_samples:
            continue  # Пропускаем последний неполный фрагмент
        sf.write(f"{output_dir}/{os.path.basename(input_path)}_{i}.wav", 
                segment, sr)
   
wav_folder_a = os.path.join(midi_folder_a, "wav")
wav_folder_b = os.path.join(midi_folder_b, "wav")
     
def split_segment(wav_folder):
    for root, dirs, files in os.walk(wav_folder):
        for file in files:
            if file.endswith('.wav'):
                file_path = os.path.join(root, file)
                split_wav(file_path, wav_folder)

def split():
    split_segment(wav_folder_a)
    split_segment(wav_folder_b)

Создаем спектрограммы:

In [ ]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, utils
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import joblib

output_dir = "preprocessed_data"
segment_duration = 1.0
max_duration_per_instrument = 5 * 60
# Здесь используем частоту дискретизации в 22050 Гц, чтобы ограничить размер спектрограмм без потери качества распознавания модели
target_sr = 22050
n_mels = 128
img_height, img_width = n_mels, 128

os.makedirs(output_dir, exist_ok=True)

def process_audio_file(file_path, label, max_segments=None):
    try:
        y, sr = librosa.load(file_path, sr=22050)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return []
    
    duration = librosa.get_duration(y=y, sr=sr)
    segment_samples = int(segment_duration * sr)
    
    segments = []
    for start in range(0, len(y), segment_samples):
        end = start + segment_samples
        if end > len(y):
            break
        segment = y[start:end]
        
        S = librosa.feature.melspectrogram(y=segment, sr=sr, n_mels=128)
        S_db = librosa.power_to_db(S, ref=np.max)
        
        segments.append((S_db, label))
    
    if max_segments is not None:
        segments = segments[:max_segments]
    
    return segments

Создаем модель

In [ ]:
import tensorflow as tf

def create_cnn_model(input_shape):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(256, (5, 5), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        layers.GlobalAveragePooling2D(),
        
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.6),
        layers.Dense(1, activation='sigmoid')
    ])
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', 'AUC', tf.keras.metrics.Recall()]
    )
    
    return model


Теперь - обучение

In [ ]:
def train_model(X_train, y_train, X_val, y_val, input_shape):

    model = create_cnn_model(input_shape)
    model.summary()
    
    callbacks = [
        EarlyStopping(patience=10, restore_best_weights=True),
        ModelCheckpoint('best_model.h5', save_best_only=True)
    ]

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=32,
        callbacks=callbacks,
    )
    
    return model, history


Используем все функции

In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.metrics import AUC
import tensorflow as tf
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import roc_curve, auc

def save_training_report(history, y_test, y_pred, y_prob, output_dir):
    report_path = os.path.join(output_dir, "training_report.pdf")
    with PdfPages(report_path) as pdf:
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Val Accuracy')
        plt.title('Accuracy')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Val Loss')
        plt.title('Loss')
        plt.legend()
        plt.tight_layout()
        pdf.savefig()
        plt.close()

        cm = confusion_matrix(y_test, y_pred)
        plt.figure()
        plt.imshow(cm, cmap='Blues')
        plt.title('Confusion Matrix')
        plt.colorbar()
        plt.xlabel('Predicted')
        plt.ylabel('True')
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                plt.text(j, i, cm[i, j], ha='center', va='center', color='red')
        pdf.savefig()
        plt.close()

        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', label=f'ROC curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        pdf.savefig()
        plt.close()

        report_text = classification_report(y_test, y_pred)
        plt.figure(figsize=(8, 5))
        plt.axis('off')
        plt.text(0.01, 0.6, report_text, fontsize=10, fontfamily='monospace')
        plt.title('Classification Report')
        pdf.savefig()
        plt.close()


def balance_samples(X, y):
    if len(X) == 0 or len(y) == 0:
        raise ValueError("Пустые данные для балансировки")
    
    X = np.array(X)
    y = np.array(y)
    
    class0_idx = np.where(y == 0)[0]
    class1_idx = np.where(y == 1)[0]
    
    if len(class0_idx) == 0 or len(class1_idx) == 0:
        raise ValueError("Один из классов отсутствует в данных")
    
    max_samples = max(len(class0_idx), len(class1_idx))
    
    if len(class0_idx) < max_samples:
        resampled_idx = resample(class0_idx, replace=True, n_samples=max_samples, random_state=42)
        balanced_idx = np.concatenate([resampled_idx, class1_idx])
    else:
        resampled_idx = resample(class1_idx, replace=True, n_samples=max_samples, random_state=42)
        balanced_idx = np.concatenate([class0_idx, resampled_idx])
    
    np.random.shuffle(balanced_idx)
    return X[balanced_idx], y[balanced_idx]

def prepare_dataset(file_list, violin_class="скрипка", violin_limit=5000, other_limit_ratio=3):
    class0_segments = []
    class1_segments = []
    
    for file_path in file_list:
        file = os.path.basename(file_path)
        label = 1 if violin_class in file.lower() else 0
        
        segments = process_audio_file(file_path, label)
        
        if label == 1:
            class1_segments.extend(segments)
        else:
            class0_segments.extend(segments)
    
    class1_segments = class1_segments[:violin_limit]
    
    class0_segments = class0_segments[:violin_limit * other_limit_ratio]
    
    all_segments = class0_segments + class1_segments
    np.random.shuffle(all_segments)
    
    X = np.array([s[0] for s in all_segments])
    y = np.array([s[1] for s in all_segments])
    return X[..., np.newaxis], y

def main():
    
    midi_to_instruments()
    midi_detele_silence()
    normalize_midi_instrumemts()
    midi_to_wav()
    split()
    
    output_dir = "output"
    
    def collect_files(folder):
        collected = []
        for root, _, files in os.walk(folder):
            for f in files:
                if f.endswith('.wav'):
                    collected.append(os.path.join(root, f))
        return collected

    train_files = collect_files(wav_folder_a)
    test_files = collect_files(wav_folder_b)

    if not train_files or not test_files:
        raise ValueError("Не найдены .wav файлы в одной из папок")

    print(f"Найдено тренировочных файлов: {len(train_files)}")
    print(f"Найдено тестовых файлов: {len(test_files)}")
    
    try:
        X_train, y_train = prepare_dataset(train_files, violin_limit=390*5)
    except Exception as e:
        raise
    
    try:
        X_test, y_test = prepare_dataset(test_files, violin_limit=1000)
    except Exception as e:
        raise
    
    try:
        X_train, y_train = balance_samples(X_train, y_train)
    except ValueError as e:
        raise
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)
        
    os.makedirs(output_dir, exist_ok=True)
    save_path = os.path.join(output_dir, 'processed_data.joblib')
    joblib.dump({
        'X_train': X_train, 'y_train': y_train,
        'X_val': X_val, 'y_val': y_val,
        'X_test': X_test, 'y_test': y_test
    }, save_path)
    
    input_shape = X_train.shape[1:]
    model, history = train_model(X_train, y_train, X_val, y_val, input_shape)
    
    y_prob = model.predict(X_test).flatten()
    y_pred = (y_prob >= 0.5).astype(int)

    save_training_report(history, y_test, y_pred, y_prob, output_dir)

    model_save_path = os.path.join(output_dir, "violin_model.h5")
    model.save(model_save_path)

if __name__ == "__main__":
    main()